In [ ]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import json
import re 
import time
from pandas.io.json import json_normalize

# 1. Data Parsing

In [ ]:
chall_game = pd.read_csv('../input/tft-match-data/TFT_Challenger_MatchData.csv')
gr_game = pd.read_csv('../input/tft-match-data/TFT_GrandMaster_MatchData.csv')
master_game = pd.read_csv('../input/tft-match-data/TFT_Master_MatchData.csv')
champ = pd.read_csv('../input/league-of-legends-tftteamfight-tacticschampion/TFT_Champion_CurrentVersion.csv')

# 2. Data Consistency check

In [ ]:
chall_game.head()

In [ ]:
gr_game.head()

In [ ]:
master_game.head()

## 2-1. Check ranking data from 1st to 8th place by game for each rank

In [ ]:
# Challenger
data_cons = chall_game.groupby('gameId')['Ranked'].count().tolist()
err_game = []

for i in range(len(data_cons)):
    if data_cons[i] != 8:
        print(chall_game.groupby('gameId')['Ranked'].count().keys()[i])
        err_game.append(chall_game.groupby('gameId')['Ranked'].count().keys()[i])

In [ ]:
# GrandMaster
data_cons = gr_game.groupby('gameId')['Ranked'].count().tolist()
err_game2 = []

for i in range(len(data_cons)):
    if data_cons[i] != 8:
        print(gr_game.groupby('gameId')['Ranked'].count().keys()[i])
        err_game2.append(gr_game.groupby('gameId')['Ranked'].count().keys()[i])

In [ ]:
# Master
data_cons = master_game.groupby('gameId')['Ranked'].count().tolist()
err_game3 = []

for i in range(len(data_cons)):
    if data_cons[i] != 8:
        print(master_game.groupby('gameId')['Ranked'].count().keys()[i])
        err_game3.append(master_game.groupby('gameId')['Ranked'].count().keys()[i])

In [ ]:
# Delete data that does not match the consistency

chall_game = chall_game[chall_game['gameId'] != err_game[0]]
master_game = master_game[master_game['gameId'] != err_game3[0]]


# 3. Analytics

## 3-0. Make Classes Synergy, Job Synergy

- Classes = type
- Job = work

In [ ]:
# Class

type_df = pd.DataFrame()
type_df['type'] = champ['origin'].unique().tolist()

type_df['1st'] = [2,3,2,3,3,3,3,2,3,2]
type_df['2nd'] = [4,6,4,6,np.nan,6,6,np.nan,np.nan,4]
type_df['3rd'] = [np.nan,np.nan,6,9,np.nan,np.nan,9,np.nan,np.nan,6]


#-----------------
#Suited Match Game data combination names
type_df['type'].iloc[4] = 'MechPilot'
type_df['type'].iloc[0] = 'SpacePirate'
type_df['type'].iloc[8] = 'Set3_Void'
type_df['type'].iloc[2] = 'Set3_Celestial'
type_df['type'].iloc[1] = 'StarGuardian'
type_df['type'].iloc[3] = 'DarkStar'



# Job

work = []
for i in champ['class'].tolist():
    work0 = i.split(',')
    for j in work0:
        work.append(''.join(re.findall('[a-zA-Z]',j)))
        
work_df = pd.DataFrame()

#------------
#for unique job name
work_df['work'] = list(set(work))[:7] + list(set(work))[8:]

work_df['1st'] = [2,2,3,2,2,1,2,2,2,1,2,2,2]
work_df['2nd'] = [4,4,6,4,4,np.nan,np.nan,np.nan,4,np.nan,4,np.nan,4]
work_df['3rd'] = [np.nan,6,9,6,np.nan,np.nan,np.nan,np.nan,6,np.nan,np.nan,np.nan,np.nan]
work_df['4th'] = [np.nan,8,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]

work_df['work'].iloc[1] = 'Set3_Sorcerer'
work_df['work'].iloc[2] = 'Set3_Blademaster'
work_df['work'].iloc[4] = 'Set3_Mystic'
work_df['work'].iloc[10] = 'Set3_Brawler'

In [ ]:
work_df

In [ ]:
type_df

- null : If there are no more synergies

## 3-1. Game Duration Check

In [ ]:
print('challenger game Duration : ', chall_game['gameDuration'].mean())
print('grandmaster game Duration : ', gr_game['gameDuration'].mean())
print('master game Duration : ', master_game['gameDuration'].mean())

- For each rank similar GameDuration 

## 3-2. Each Ranked combination distribution

- challenger, grandmaster, master game combination distribution
    - first place distribution
    - upper, Subvolume distribution

In [ ]:
# function

def combination_distribution(df, rank, rank_range = False):
    
    if rank_range == False:
        df = df[df['Ranked']==rank]
    elif rank_range == True:
        
        df2 = pd.DataFrame()
        
        for i in range(rank[0],rank[1]+1):
            df2 = df2.append(df[df['Ranked']==i])
            
        df = df2
    
    forth_combi2 = []
    third_combi, third_combi2 = [],[]
    second_combi, second_combi2 = [],[]
    first_combi, first_combi2 = [],[]
    non_combi, non_combi2 = [], []
    temp_ls = []

    # 전체등수에 대해서
    for i in range(len(df)):
        char = df['combination'].iloc[i]
        char2 = char.replace("'","\"")
        key_ls = list(json.loads(char2).keys())
        value_ls = list(json.loads(char2).values())
        if i != 0  and i % 1000 ==0:
            print(i)

        for j in range(len(key_ls)):

            if key_ls[j] in type_df['type'].tolist():
                #마지막 시너지를 내고 있는 계열
                if type_df[type_df['type']==key_ls[j]]['3rd'].iloc[0] == np.nan:
                    pass #3rd가 null값이면 2nd나 1st에서 마지막 시너지이므로 패스
                else:
                    if value_ls[j] >= type_df[type_df['type']==key_ls[j]]['3rd'].iloc[0]:
                        third_combi.append(key_ls[j])#3rd 시너지를 내고 있는 것
                        continue
                    else:
                        pass

                if type_df[type_df['type']==key_ls[j]]['2nd'].iloc[0] == np.nan:
                    pass
                else:
                    if value_ls[j] >= type_df[type_df['type']==key_ls[j]]['2nd'].iloc[0]:
                        second_combi.append(key_ls[j])
                        continue
                    else:
                        pass

                if value_ls[j] >= type_df[type_df['type']==key_ls[j]]['1st'].iloc[0]:
                    first_combi.append(key_ls[j])

                else:
                    non_combi.append(key_ls[j])


            elif key_ls[j] in work_df['work'].tolist():


                if work_df[work_df['work']==key_ls[j]]['4th'].iloc[0] == np.nan:
                    pass #3rd가 null값이면 2nd나 1st에서 마지막 시너지이므로 패스

                else:
                    if value_ls[j] >= work_df[work_df['work']==key_ls[j]]['4th'].iloc[0]:
                        forth_combi2.append(key_ls[j]) #4th 시너지를 내고 있는 것
                        continue
                    else:
                        pass

                if work_df[work_df['work']==key_ls[j]]['3rd'].iloc[0] == np.nan:
                    pass
                else:
                    if value_ls[j] >= work_df[work_df['work']==key_ls[j]]['3rd'].iloc[0]:
                        third_combi2.append(key_ls[j])
                        continue
                    else:
                        pass

                if work_df[work_df['work']==key_ls[j]]['2nd'].iloc[0] == np.nan:
                    pass
                else:
                    if value_ls[j] >= work_df[work_df['work']==key_ls[j]]['2nd'].iloc[0]:
                        second_combi2.append(key_ls[j])
                        continue
                    else:
                        pass

                if value_ls[j] >= work_df[work_df['work']==key_ls[j]]['1st'].iloc[0]:
                    first_combi2.append(key_ls[j])

                else:
                    non_combi2.append(key_ls[j])


            else:
                temp_ls.append(i)
                
    return non_combi, non_combi2, first_combi, first_combi2, second_combi, second_combi2\
            ,third_combi, third_combi2, forth_combi2, temp_ls

- non_combi : non classes Synergy
- non_combi2 : non Job synergy

- first_combi : Step 1 classes Synergy
- fitst_combi2 : Step 1 Job Synergy

- second_combi : Step 2 classes Synergy
- second_combi2 : Step 2 Job Synergy

- third_combi : Step 3 classes Synergy
- third_combi2 : Step 3 Job Synergy

- forth_combi2 : Step 4 Job Synergy

- temp_ls : TemplateTrait(Skill)


**Even if the first thing you need to know is that there are careers and classes that produce final synergy (for example, mercenaries, spaceships, voids, valkyrie, etc.), we will not cover those parts yet and will be covered below.**

**Therefore, a combination of mercenaries, valkyrie, and voids is included in the first-level synergy.**



In [ ]:
# challenger first place distribution

non_combi, non_combi2, first_combi, first_combi2, second_combi, second_combi2,third_combi, third_combi2, forth_combi2, temp_ls = combination_distribution(chall_game,1)

In [ ]:
# grand master first place distribution
gr_non_combi, gr_non_combi2, gr_first_combi, gr_first_combi2, gr_second_combi, gr_second_combi2,gr_third_combi, gr_third_combi2, gr_forth_combi2, gr_temp_ls = combination_distribution(gr_game,1)

In [ ]:
# master first place distribution
mas_non_combi, mas_non_combi2, mas_first_combi, mas_first_combi2,mas_second_combi, mas_second_combi2,mas_third_combi, mas_third_combi2, mas_forth_combi2, mas_temp_ls = combination_distribution(master_game,1)

## 3-3. EDA

In [ ]:
def class_distribution_plot(first_combination_list, second_combination_list, third_combination_list):

    fig,axes = plt.subplots(3,1,figsize = (12,36))
    
    plot_key0 = pd.Series(first_combination_list).value_counts().keys().tolist()
    plot_value0 = pd.Series(first_combination_list).value_counts().values.tolist()
    
    plot_key1 = pd.Series(second_combination_list).value_counts().keys().tolist()
    plot_value1 = pd.Series(second_combination_list).value_counts().values.tolist()
    
    plot_key2 = pd.Series(third_combination_list).value_counts().keys().tolist()
    plot_value2 = pd.Series(third_combination_list).value_counts().values.tolist()
    

    for ls,ax in zip([[plot_key0,plot_value0],[plot_key1,plot_value1],[plot_key2,plot_value2]],axes):
        
        ax.barh(ls[0],ls[1])

        #ax.set_title('{} 이웃'.format(n_neighbors))
        ax.set_xlabel('Value')
        ax.set_ylabel('Synergy')
        
def job_distribution_plot(first_combination_list, second_combination_list, third_combination_list,forth_combination):

    fig,axes = plt.subplots(4,1,figsize = (12,48))
    
    plot_key0 = pd.Series(first_combination_list).value_counts().keys().tolist()
    plot_value0 = pd.Series(first_combination_list).value_counts().values.tolist()
    
    plot_key1 = pd.Series(second_combination_list).value_counts().keys().tolist()
    plot_value1 = pd.Series(second_combination_list).value_counts().values.tolist()
    
    plot_key2 = pd.Series(third_combination_list).value_counts().keys().tolist()
    plot_value2 = pd.Series(third_combination_list).value_counts().values.tolist()
    
    plot_key3 = pd.Series(forth_combination).value_counts().keys().tolist()
    plot_value3 = pd.Series(forth_combination).value_counts().values.tolist()
    

    for ls,ax in zip([[plot_key0,plot_value0],[plot_key1,plot_value1],[plot_key2,plot_value2],[plot_key3,plot_value3]],axes):
        
        ax.barh(ls[0],ls[1])

        #ax.set_title('{} 이웃'.format(n_neighbors))
        ax.set_xlabel('Value')
        ax.set_ylabel('Synergy')
    

### 3-3-1. Challenger

In [ ]:
class_distribution_plot(first_combi,second_combi,third_combi)

- Challenger Classes Distribution of preferred combinations for each stage of each game
    - Step 1
        - best : Chrono / Set3_Celestial / Valkyrie (Final Combination: Valkyrie)
        - worst : Cybernetic / Set3_Void / StarGuardian (Final Combination: Set3_Void)
            
    - Step 2
        - best : Cybernetic / Set3_Celestial / StarGuardian (Final Combination: Cybernetic, StarGuardian)
        - worst : SpacePirate / Rebel (Final Combination : SpacePirate)
            
    - Step 3
        - best : Set3_Celestial (Final Combination : Set3_Celestial)
        - worst : Rebel (Final Combination : Rebel)
            
*Final Combination : No more champions to combine

- Very favorite classes group is Set3_Celestial
- Star Guardians and Cybernetics rarely match the first stage, and prefer the second stage or higher.
- The rebels are largely unfavorable.

In [ ]:
job_distribution_plot(first_combi2,second_combi2,third_combi2,forth_combi2)

- Challenger Job Distribution of preferred combinations for each stage of each game
    - Step 1
        - best : Infiltrator / Set3_Mystic / Set3_Bladmaster (Final Combination: X)
        - worst : Mercenary / Set3_Brawler / Protector (Final Combination: Mercenary)
            
    - Step 2
        - best : Set3_Brawler / Set3_Sorcerer (Final Combination: Set3_Brawler)
        - worst : Demolitionist / Set3_Bladmaster (Final Combination : Demolitionist)
            
    - Step 3
        - best : Set3_Sorcerer (Final Combination : X)
        - worst : Set3_Bladmaster (Final Combination : Set3_Bladmaster)
    
    - Step 4
        - only one : Set3_Sorcerer
            
*Final Combination : No more champions to combine

- First-class users never go to spaceships with job synergy.
- Also The infiltrator prefers only the first stage.
- Mana predators are also not preferred.
- Preferred occupations Set3_Brawler and sorcerers, and mystics usually prefer only up to Level 1.

### 3-3-2. Grand Master

In [ ]:
class_distribution_plot(gr_first_combi,gr_second_combi,gr_third_combi)

In [ ]:
job_distribution_plot(gr_first_combi2,gr_second_combi2,gr_third_combi2,gr_forth_combi2)

### 3-3-3. Master

In [ ]:
class_distribution_plot(mas_first_combi,mas_second_combi,mas_third_combi)

In [ ]:
job_distribution_plot(mas_first_combi2,mas_second_combi2,mas_third_combi2,mas_forth_combi2)

## 3-4. Which champions you prefer for each combination

## 3-5. Distribution of upper and lower volumes